As of MLflow 2.9.0, Model Stages have been deprecated and will be removed in a future major release (https://www.mlflow.org/docs/latest/model-registry.html#migrating-from-stages)
Summary: Stages are rpaced with aliases now. It means that instead of "models:/regression_model/Production" there should be something like: "models:/prod.ml_team.regression_model@champion"

In [16]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

list_experiments is depricated

In [17]:
client.search_experiments()
# client.get_experiment_by_name("nyc-taxi-experiment").experiment_id

[<Experiment: artifact_location='/home/nikto/study/mlops-zoomcamp/02-experiment-tracking/mlruns/3', creation_time=1721833580704, experiment_id='3', last_update_time=1721833580704, lifecycle_stage='active', name='my-kool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1721216752638, experiment_id='2', last_update_time=1721216752638, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1720444747090, experiment_id='0', last_update_time=1720444747090, lifecycle_stage='active', name='Default', tags={}>]

client.create_experiment(name='my-kool-experiment')

In [57]:
runs = client.search_runs(
    experiment_ids= client.get_experiment_by_name("nyc-taxi-experiment").experiment_id,
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=['metrics.rmse ASC'],
    filter_string="attributes.status = 'FINISHED'"
    )

In [126]:
for run in runs:
    print(f"Run ID: {run.info.run_id}, Status: {run.info.status}, RMSE: {run.data.metrics['rmse']:.2f}")

Run ID: df1a438d21064d82bffea8198b20b005, Status: FINISHED, RMSE: 6.30
Run ID: b36cb4007ec04d68a7220e6daf0e5c35, Status: FINISHED, RMSE: 6.32
Run ID: 4e1510515af0483d80095b7c21d1dff0, Status: FINISHED, RMSE: 6.32
Run ID: 3428fbeb84a0469daa086435a494cafd, Status: FINISHED, RMSE: 6.32
Run ID: 631ae7a02cbd43fda1f7a3d2c134e4fd, Status: FINISHED, RMSE: 6.32
Run ID: 5956783bfc9a4f8192a2710792199af1, Status: FINISHED, RMSE: 6.32
Run ID: 7dad4588d5274d3bbd2ca1a911ad0975, Status: FINISHED, RMSE: 6.32
Run ID: b5eba2b688c7406eb845b01a018ba3cf, Status: FINISHED, RMSE: 6.32
Run ID: d55e5ddc3dcd4e1dbff8ba06b2c17e57, Status: FINISHED, RMSE: 6.33
Run ID: ccaccfb9dbe64331890526c84107bc20, Status: FINISHED, RMSE: 6.33


In [60]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [95]:
run_id='3428fbeb84a0469daa086435a494cafd'
model_uri=f'runs:/{run_id}/model'
mlflow.register_model(model_uri=model_uri,name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1721839886289, current_stage='None', description=None, last_updated_timestamp=1721839886289, name='nyc-taxi-regressor', run_id='3428fbeb84a0469daa086435a494cafd', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/3428fbeb84a0469daa086435a494cafd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

and again, list_registered_models is depricated

In [63]:
client.search_registered_models()

[<RegisteredModel: aliases={'verstion2': 2}, creation_timestamp=1721807937105, description='', last_updated_timestamp=1721837647713, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1721837647713, current_stage='None', description=None, last_updated_timestamp=1721837647713, name='nyc-taxi-regressor', run_id='3428fbeb84a0469daa086435a494cafd', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/3428fbeb84a0469daa086435a494cafd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [81]:
model_name='nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)


/tmp/ipykernel_261933/1683035849.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [79]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage='Staging',
    archive_existing_versions=True
)

/tmp/ipykernel_261933/1980899186.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1721837647713, current_stage='Staging', description=None, last_updated_timestamp=1721838405392, name='nyc-taxi-regressor', run_id='3428fbeb84a0469daa086435a494cafd', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/3428fbeb84a0469daa086435a494cafd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [82]:
for version in latest_versions:
    print(f'Version: {version.version}, stage: {version.current_stage}')

Version: 2, stage: None
Version: 3, stage: Staging


test_model updated to work with aliases

In [113]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, X_test, y_test, stage=None, alias=None):
    if stage:
        model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    elif alias:
        model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    else:
        raise Exception('Define stage or alias')
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [92]:
dataframe_url='https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet'

In [94]:
df=read_dataframe(dataframe_url)

In [99]:
client.download_artifacts(run_id='7dad4588d5274d3bbd2ca1a911ad0975',path='preprocessor',dst_path='.')

'/home/nikto/study/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [105]:
import pickle
with open('preprocessor/preprocessor.b','rb') as f_in:
    dv = pickle.load(f_in)

In [106]:
X_test = preprocess(df,dv)

In [108]:
target = 'duration'
y_test = df[target].values

In [116]:
%time test_model(name=model_name,alias='version1',X_test=X_test,y_test=y_test)

CPU times: user 9.76 s, sys: 34.3 ms, total: 9.79 s
Wall time: 910 ms


/home/nikto/study/mlops-zoomcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.270748587173265}

In [120]:
%time test_model(name=model_name,alias='version2',X_test=X_test,y_test=y_test)

CPU times: user 9.11 s, sys: 0 ns, total: 9.11 s
Wall time: 615 ms


/home/nikto/study/mlops-zoomcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.270748587173265}